<a href="https://colab.research.google.com/github/laurencoetzee001/Beads_Co-detect/blob/main/prompt_optimisation_location_exchange_revisions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Bead Trade Coding System - 27,000 Rows Google Colab PRODUCTION
==============================================================

COMPLETE IMPLEMENTATION WITH ALL REVISIONS:
✓ 13-field structure (codes + descriptions for research value)
✓ JSON dual-format output (maintained from your working version)
✓ Enhanced 4a_exchange rules (73% Human-AI agreement)
  - Expert-validated edge cases (23:1 over-identification fix)
  - Checkmarks/visual anchors for LLM parsing
  - Conservative "when in doubt, code NO"
✓ Two-layer location extraction (NEW)
  - 8_location_names_original_spellings (preserve exact historical spellings)
  - 8_location_context (geographic + strategic + political context)
✓ Data quality pre-checks (filter corrupted OCR, short entries)
✓ 60% token reduction in prompting (~800 tokens/row)
✓ 2,000-row batches with 50-row checkpoints
✓ Robust Google Drive persistence

27,000 rows across ~14 sessions at 800K TPM
"""

import subprocess
import sys
import os
import json
import time
from datetime import datetime

# === INSTALL DEPENDENCIES ===
print("Checking dependencies...")
for pkg in ["anthropic", "openpyxl", "pandas", "tenacity"]:
    try:
        __import__(pkg if pkg != "openpyxl" else "openpyxl")
    except ImportError:
        print(f"Installing {pkg}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "-q"])

import pandas as pd
from anthropic import Anthropic
from tenacity import retry, stop_after_attempt, wait_exponential

# === GOOGLE DRIVE SETUP ===
try:
    from google.colab import drive
    drive.mount('/content/drive')
    GDRIVE_BASE = '/content/drive/MyDrive/bead_coding_27k_final'
    os.makedirs(GDRIVE_BASE, exist_ok=True)
    print("✓ Google Drive mounted")
except:
    GDRIVE_BASE = './bead_coding_27k_final'
    os.makedirs(GDRIVE_BASE, exist_ok=True)
    print("⚠ Running locally (not in Colab)")

# === CONFIGURATION ===
INPUT_FILE = "All_entries_beads_cleaned.xlsx"
OUTPUT_BASE = GDRIVE_BASE
LOG_FILE = os.path.join(OUTPUT_BASE, "processing.log")
STATE_FILE = os.path.join(OUTPUT_BASE, "session_state.json")

MODEL_NAME = "claude-haiku-4-5-20251001"
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY", "")
MAX_SESSION_MINUTES = 690  # 11.5 hours buffer
TEXT_COLUMN = "text_page_gp"
ROWS_PER_BATCH = 2000
SAVE_EVERY = 50

# === OPTIMIZED SYSTEM PROMPT WITH ALL REVISIONS ===
SYSTEM_PROMPT = """You are a historian analyzing pre-colonial African bead trade records.
Apply expert-validated codebook v4.0 with enhanced location extraction.

TARGET: 73% human-AI agreement rate through conservative interpretation.

RESPONSE: Return ONLY JSON. NO markdown, NO ```json``` tags, NO extra text.

=== DATA QUALITY PRE-CHECK ===
Assess text quality FIRST:
- <50 readable characters → {"quality_issue": "too_short"}
- Mostly symbols/OCR errors → {"quality_issue": "corrupted_ocr"}
- No bead content → {"quality_issue": "no_bead_content"}
- Otherwise → proceed with full 15-field coding

=== CRITICAL: 4A_EXCHANGE DECISION RULES ===

✓ CODE "xo" (EXCHANGE OCCURRED) ONLY IF ALL PRESENT:
  ✓ Transaction verb: traded, sold, bought, exchanged, gave, received
  ✓ Past tense: action already happened
  ✓ Clear parties: who gave/received
  ✓ Specific items: what was exchanged

✗ CODE "no" (NO EXCHANGE) FOR:
  ✗ Value descriptions: "beads are worth...", "beads cost..."
  ✗ Manufacturing: "making", "creating", "fashioned"
  ✗ Hypothetical/future: "would trade", "could buy"
  ✗ General statements: "natives sell ivory for beads"
  ✗ Observational: "showed beads", "wore beads", "displayed"
  ✗ Equipment/decoration: saddlery, clothing, adornment

EXPERT-VALIDATED EDGE CASES:
  → Gift-giving ("offered presents") = xo (valid transaction)
  → Intangible ("bought secret for beads") = xo (still transaction)
  → Historical generalizations = no (not specific event)
  → Observational contexts = no (not exchange)
  → Equipment beads = no (decorative, not traded)

=== TWO-LAYER LOCATION EXTRACTION ===

LAYER 1: 8_LOCATION_NAMES_ORIGINAL_SPELLINGS
- Comma-separated list of EXACT place names as written in text
- Preserve diacritics/accents: Bontúku, Dahomey, Whydah (NOT normalized)
- Include parenthetical geographic markers if original text has them
- Order by first mention
- Research value: Historical placename variants, etymology

LAYER 2: 8_LOCATION_CONTEXT
- For each location, capture geographic + strategic + political context
- Format: Location | Geographic Context | Strategic Significance | Other Details
- Examples:
  • "Bontúku | near Kumasi | distribution center for Akan trade | Ashanti territory"
  • "Cape Coast | Gold Coast, coastal settlement | European trading fort | Atlantic hub"
- Research value: Trade route mapping, network analysis, merchant preferences

=== JSON STRUCTURE (15 fields) ===

{
  "quality_check": "pass" or "quality_issue description",
  "1_price_HUMAN": {
    "status": "yes|no|xo",
    "amount": null or "number/measurement",
    "currency": null or "currency/commodity",
    "description": "full price text from source"
  },
  "2_size_HUMAN": {
    "code": null or 1-6,
    "description": "exact size text"
  },
  "3_colour_HUMAN": {
    "codes": [],
    "description": "exact color text"
  },
  "4_location_HUMAN": {
    "codes": [],
    "names": "actual location names"
  },
  "5_function_HUMAN": {
    "codes": [],
    "description": "detailed function text"
  },
  "6_origin_of_bead": "geographic origin text or null",
  "7_shape_HUMAN": {
    "codes": [],
    "description": "exact shape text"
  },
  "8_type_bead_HUMAN": {
    "codes": [],
    "description": "exact material text"
  },
  "8_location_names_original_spellings": "Bontúku, Whydah, etc. (exact spellings)",
  "8_location_context": "Location | Geographic | Strategic | Other details",
  "9_local_name_HUMAN": {
    "exists": null or 1|2,
    "names": null or ["array of names"]
  },
  "10_relationship_": {
    "codes": [],
    "description": "detailed exchange items text"
  },
  "11_units_of_measure": {
    "type": null or 1-4,
    "description": "exact measurement text"
  },
  "12_bead_ethnic_": ["array of ethnic group names"] or null,
  "13_nature_of_exchange": {
    "code": null or 1-6,
    "description": "exchange nature text"
  },
  "notes": "additional research context"
}

=== PROCESSING RULES ===
1. Use null for missing/ambiguous data (not "unknown")
2. For arrays: [] if no data, null if not applicable
3. ALWAYS include description fields with verbatim text
4. When code=14 (other), description is MANDATORY
5. Preserve exact terminology for research value
6. Conservative interpretation: when in doubt, code conservative
"""

USER_PROMPT_TEMPLATE = """Analyze and extract ALL 15 fields (with quality check):

TEXT:
{text}

Return ONLY the JSON object. No ```json``` tags, no other text."""

# === SESSION STATE MANAGEMENT ===
class SessionState:
    def __init__(self):
        self.file_path = STATE_FILE
        self.state = self._load()

    def _load(self):
        if os.path.exists(self.file_path):
            with open(self.file_path, 'r') as f:
                return json.load(f)
        return {
            'session_num': 1,
            'last_row': 0,
            'sessions_completed': 0,
            'total_tokens_used': 0,
            'total_cost': 0,
            'batches_completed': []
        }

    def save(self):
        with open(self.file_path, 'w') as f:
            json.dump(self.state, f, indent=2)

    def get_batch_info(self, total_rows=27000):
        session_num = self.state['session_num']
        start_row = (session_num - 1) * ROWS_PER_BATCH
        end_row = min(session_num * ROWS_PER_BATCH, total_rows)

        if self.state['last_row'] >= start_row:
            actual_start = self.state['last_row'] + 1
        else:
            actual_start = start_row

        return {
            'session': session_num,
            'batch_start': start_row,
            'batch_end': end_row,
            'actual_start': actual_start,
            'batch_size': end_row - start_row
        }

    def update_row(self, row_num):
        self.state['last_row'] = row_num
        self.save()

    def mark_batch_complete(self, total_input_tokens, total_output_tokens, cost):
        self.state['sessions_completed'] += 1
        self.state['session_num'] += 1
        self.state['total_tokens_used'] += total_input_tokens + total_output_tokens
        self.state['total_cost'] += cost
        self.state['batches_completed'].append({
            'session': self.state['sessions_completed'],
            'rows': f"{self.state['last_row']-ROWS_PER_BATCH+1}-{self.state['last_row']}",
            'tokens': total_input_tokens + total_output_tokens,
            'cost': cost
        })
        self.save()

# === UTILITY FUNCTIONS ===

def strip_markdown_json(text):
    """Remove markdown and extract JSON object."""
    text = text.strip()

    if text.startswith('```'):
        lines = text.split('\n')
        if lines[0].strip().startswith('```'):
            lines = lines[1:]
        if lines and lines[-1].strip() == '```':
            lines = lines[:-1]
        text = '\n'.join(lines).strip()

    if not text.startswith('{'):
        start = text.find('{')
        if start == -1:
            return text
        text = text[start:]

    brace_count = 0
    in_string = False
    escape_next = False

    for i, char in enumerate(text):
        if escape_next:
            escape_next = False
            continue
        if char == '\\':
            escape_next = True
            continue
        if char == '"':
            in_string = not in_string
            continue
        if not in_string:
            if char == '{':
                brace_count += 1
            elif char == '}':
                brace_count -= 1
                if brace_count == 0:
                    return text[:i+1]

    return text

def validate_json_response(json_obj):
    """Validate required fields exist."""
    required = [
        '1_price_HUMAN', '2_size_HUMAN', '3_colour_HUMAN', '4_location_HUMAN',
        '5_function_HUMAN', '6_origin_of_bead', '7_shape_HUMAN', '8_type_bead_HUMAN',
        '9_local_name_HUMAN', '10_relationship_', '11_units_of_measure',
        '12_bead_ethnic_', '13_nature_of_exchange'
    ]

    missing = [f for f in required if f not in json_obj]
    if missing:
        return False, f"Missing: {', '.join(missing[:3])}"

    # Check new location fields
    if '8_location_names_original_spellings' not in json_obj or '8_location_context' not in json_obj:
        return False, "Missing location enhancement fields"

    return True, None

def flatten_for_excel(json_obj):
    """Flatten nested JSON to Excel format (29 columns with locations)."""
    flat = {}

    # Quality check
    flat['quality_check'] = json_obj.get('quality_check', 'pass')

    # 1_price_HUMAN (4 columns)
    price = json_obj.get('1_price_HUMAN', {})
    if isinstance(price, dict):
        flat['1_price_status'] = price.get('status')
        flat['1_price_amount'] = price.get('amount')
        flat['1_price_currency'] = price.get('currency')
        flat['1_price_description'] = price.get('description')

    # 2_size_HUMAN (2 columns)
    size = json_obj.get('2_size_HUMAN', {})
    if isinstance(size, dict):
        flat['2_size_code'] = size.get('code')
        flat['2_size_description'] = size.get('description')

    # 3_colour_HUMAN (2 columns)
    color = json_obj.get('3_colour_HUMAN', {})
    if isinstance(color, dict):
        codes = color.get('codes', [])
        flat['3_colour_codes'] = ','.join(map(str, codes)) if codes else None
        flat['3_colour_description'] = color.get('description')

    # 4_location_HUMAN (2 columns)
    location = json_obj.get('4_location_HUMAN', {})
    if isinstance(location, dict):
        codes = location.get('codes', [])
        flat['4_location_codes'] = ','.join(map(str, codes)) if codes else None
        flat['4_location_names'] = location.get('names')

    # 5_function_HUMAN (2 columns)
    function = json_obj.get('5_function_HUMAN', {})
    if isinstance(function, dict):
        codes = function.get('codes', [])
        flat['5_function_codes'] = ','.join(map(str, codes)) if codes else None
        flat['5_function_description'] = function.get('description')

    # 6_origin_of_bead (1 column)
    flat['6_origin_of_bead'] = json_obj.get('6_origin_of_bead')

    # 7_shape_HUMAN (2 columns)
    shape = json_obj.get('7_shape_HUMAN', {})
    if isinstance(shape, dict):
        codes = shape.get('codes', [])
        flat['7_shape_codes'] = ','.join(map(str, codes)) if codes else None
        flat['7_shape_description'] = shape.get('description')

    # 8_type_bead_HUMAN (2 columns)
    bead_type = json_obj.get('8_type_bead_HUMAN', {})
    if isinstance(bead_type, dict):
        codes = bead_type.get('codes', [])
        flat['8_type_codes'] = ','.join(map(str, codes)) if codes else None
        flat['8_type_description'] = bead_type.get('description')

    # NEW: Two-layer location extraction (2 columns)
    flat['8_location_names_original_spellings'] = json_obj.get('8_location_names_original_spellings')
    flat['8_location_context'] = json_obj.get('8_location_context')

    # 9_local_name_HUMAN (2 columns)
    local = json_obj.get('9_local_name_HUMAN', {})
    if isinstance(local, dict):
        flat['9_local_name_exists'] = local.get('exists')
        names = local.get('names', [])
        flat['9_local_name_names'] = '; '.join(names) if names else None

    # 10_relationship_ (2 columns)
    rel = json_obj.get('10_relationship_', {})
    if isinstance(rel, dict):
        codes = rel.get('codes', [])
        flat['10_relationship_codes'] = ','.join(map(str, codes)) if codes else None
        flat['10_relationship_description'] = rel.get('description')

    # 11_units_of_measure (2 columns)
    units = json_obj.get('11_units_of_measure', {})
    if isinstance(units, dict):
        flat['11_units_type'] = units.get('type')
        flat['11_units_description'] = units.get('description')

    # 12_bead_ethnic_ (1 column)
    ethnics = json_obj.get('12_bead_ethnic_', [])
    flat['12_bead_ethnic_'] = '; '.join(ethnics) if ethnics else None

    # 13_nature_of_exchange (2 columns)
    nature = json_obj.get('13_nature_of_exchange', {})
    if isinstance(nature, dict):
        flat['13_nature_code'] = nature.get('code')
        flat['13_nature_description'] = nature.get('description')

    # Notes (1 column)
    flat['notes'] = json_obj.get('notes')

    return flat

@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=2, min=1, max=10),
    reraise=True
)
def call_claude(client, entry_text):
    """Call Claude API with expert-validated codebook."""
    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2500,
        temperature=0,
        system=SYSTEM_PROMPT,
        messages=[{
            "role": "user",
            "content": USER_PROMPT_TEMPLATE.format(text=entry_text)
        }]
    )
    return response

def calculate_cost(input_tokens, output_tokens):
    """Calculate API cost for Haiku."""
    input_cost = (input_tokens / 1_000_000) * 0.80
    output_cost = (output_tokens / 1_000_000) * 0.24
    return input_cost + output_cost

def log_event(message, level="INFO"):
    """Log to file and print."""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_message = f"[{timestamp}] [{level}] {message}"
    print(log_message)
    with open(LOG_FILE, 'a') as f:
        f.write(log_message + "\n")

# === MAIN PROCESSING ===

def main():
    print("\n" + "="*80)
    print("BEAD TRADE CODING - 27,000 ROWS PRODUCTION (WITH ALL REVISIONS)")
    print("="*80)
    print("✓ 13-field structure + 2-layer location extraction")
    print("✓ Enhanced 4a_exchange rules (73% Human-AI agreement)")
    print("✓ Data quality pre-checks")
    print("✓ 60% token optimization")
    print("✓ 2,000-row batches | 50-row checkpoints")

    if not ANTHROPIC_API_KEY:
        print("ERROR: ANTHROPIC_API_KEY environment variable not set")
        return

    session = SessionState()
    batch_info = session.get_batch_info()
    total_batches = (27000 + ROWS_PER_BATCH - 1) // ROWS_PER_BATCH

    print(f"\nSession Information:")
    print(f"  Current Session: {batch_info['session']}/{total_batches}")
    print(f"  Batch Range: Rows {batch_info['batch_start']}-{batch_info['batch_end']-1}")
    print(f"  Starting From: Row {batch_info['actual_start']}")
    print(f"  Checkpoint Frequency: Every {SAVE_EVERY} rows")

    log_event(f"Session {batch_info['session']}/{total_batches} started - resuming from row {batch_info['actual_start']}")

    print(f"\nLoading data file...")
    try:
        df = pd.read_excel(INPUT_FILE)
        print(f"✓ Loaded {len(df)} rows, {len(df.columns)} columns")
        log_event(f"Data loaded: {len(df)} rows")
    except Exception as e:
        print(f"ERROR: Could not load file: {e}")
        log_event(f"ERROR: Failed to load file: {e}", "ERROR")
        return

    if TEXT_COLUMN not in df.columns:
        print(f"ERROR: Column '{TEXT_COLUMN}' not found")
        return

    client = Anthropic(api_key=ANTHROPIC_API_KEY)
    responses = []
    total_input_tokens = 0
    total_output_tokens = 0
    session_start_time = time.time()
    success_count = 0
    error_count = 0
    skipped_count = 0
    quality_issues = 0
    checkpoint_count = 0

    print(f"\nStarting processing...")
    print(f"Model: {MODEL_NAME}, Temperature: 0")
    print(f"Codebook: Expert-validated v4.0 with location enhancements")
    print("-" * 80)

    for row_idx in range(batch_info['actual_start'], batch_info['batch_end']):
        elapsed_minutes = (time.time() - session_start_time) / 60
        if elapsed_minutes > MAX_SESSION_MINUTES:
            print(f"\n⏰ TIME LIMIT APPROACHING ({elapsed_minutes:.0f}m used)")
            log_event(f"Session ended at time limit: {elapsed_minutes:.0f}m")
            break

        try:
            row = df.iloc[row_idx]
            entry_text = row.get(TEXT_COLUMN)

            if pd.isna(entry_text) or not str(entry_text).strip():
                responses.append(None)
                skipped_count += 1
                continue

            entry_text = str(entry_text).strip()

            # Call Claude with enhanced codebook
            response = call_claude(client, entry_text)
            total_input_tokens += response.usage.input_tokens
            total_output_tokens += response.usage.output_tokens
            response_text = response.content[0].text

            # Strip markdown and parse
            cleaned = strip_markdown_json(response_text)

            if not cleaned:
                responses.append({"error": "Empty response"})
                error_count += 1
                continue

            try:
                parsed = json.loads(cleaned)

                # Check for quality issues
                if parsed.get('quality_check') != 'pass':
                    quality_issues += 1
                    responses.append({"quality_issue": parsed.get('quality_check')})
                    continue

                # Validate
                valid, error = validate_json_response(parsed)
                if valid:
                    flat = flatten_for_excel(parsed)
                    responses.append(flat)
                    success_count += 1
                else:
                    responses.append({"error": error})
                    error_count += 1
                    log_event(f"Row {row_idx}: Validation failed - {error}")

            except json.JSONDecodeError as e:
                responses.append({"error": f"JSON error"})
                error_count += 1
                log_event(f"Row {row_idx}: JSON parse error")

            # Update state and checkpoint every 50 rows
            session.update_row(row_idx)

            rows_in_batch = row_idx - batch_info['actual_start'] + 1
            if rows_in_batch % SAVE_EVERY == 0:
                checkpoint_count += 1
                elapsed = (time.time() - session_start_time) / 60
                cost_so_far = calculate_cost(total_input_tokens, total_output_tokens)
                avg_tokens = (total_input_tokens + total_output_tokens) / rows_in_batch if rows_in_batch > 0 else 0
                print(f"Checkpoint {checkpoint_count} (Row {row_idx}): Success={success_count}, "
                      f"Quality_issues={quality_issues}, Tokens/row={avg_tokens:.0f}, "
                      f"Cost=${cost_so_far:.2f}, Time={elapsed:.0f}m")

        except Exception as e:
            responses.append({"error": str(e)[:100]})
            error_count += 1
            session.update_row(row_idx)
            log_event(f"Row {row_idx}: Exception - {str(e)[:100]}", "ERROR")

    # === SAVE RESULTS ===

    final_cost = calculate_cost(total_input_tokens, total_output_tokens)

    print("\n" + "="*80)
    print("SESSION COMPLETE")
    print("="*80)

    # Create output dataframe
    batch_df = df.iloc[batch_info['actual_start']:batch_info['actual_start']+len(responses)].copy()

    # Add coding columns
    for i, resp in enumerate(responses):
        if resp and isinstance(resp, dict) and "error" not in resp and "quality_issue" not in resp:
            for key, value in resp.items():
                if key not in batch_df.columns:
                    batch_df[key] = None
                batch_df.at[i, key] = value

    # Save batch output
    batch_num = batch_info['session']
    output_file = os.path.join(
        OUTPUT_BASE,
        f"batch_{batch_num:02d}_rows_{batch_info['actual_start']}-{session.state['last_row']}.xlsx"
    )
    batch_df.to_excel(output_file, index=False)

    # Print summary
    print(f"\nBatch {batch_num}/{total_batches} Summary:")
    print(f"  Rows processed: {batch_info['actual_start']}-{session.state['last_row']}")
    print(f"  Successfully coded: {success_count}")
    print(f"  Quality issues: {quality_issues}")
    print(f"  Errors: {error_count}")
    print(f"  Skipped (empty): {skipped_count}")
    print(f"  Checkpoints saved: {checkpoint_count}")
    print(f"  Input tokens: {total_input_tokens:,}")
    print(f"  Output tokens: {total_output_tokens:,}")
    print(f"  Avg tokens/row: {(total_input_tokens+total_output_tokens)/(success_count if success_count > 0 else 1):.0f}")
    print(f"  Session cost: ${final_cost:.2f}")
    print(f"  Total cost so far: ${session.state['total_cost'] + final_cost:.2f}")
    print(f"  Processing time: {(time.time() - session_start_time)/60:.1f}m")
    print(f"\n  Output file: {os.path.basename(output_file)}")
    print(f"  Output columns: {len(batch_df.columns)} (original: {len(df.columns)}, new: {len(batch_df.columns) - len(df.columns)})")

    session.mark_batch_complete(total_input_tokens, total_output_tokens, final_cost)

    total_rows = 27000
    rows_completed = session.state['last_row'] + 1

    print(f"\nOverall Progress:")
    print(f"  Rows completed: {rows_completed}/{total_rows} ({(rows_completed/total_rows)*100:.1f}%)")
    print(f"  Sessions completed: {session.state['sessions_completed']}/{total_batches}")
    print(f"  Total cost: ${session.state['total_cost']:.2f}")

    if session.state['sessions_completed'] < total_batches:
        next_start = session.state['last_row'] + 1
        print(f"\n⭐️ NEXT SESSION:")
        print(f"  Will resume from row {next_start}")
    else:
        print(f"\n✅ ALL BATCHES COMPLETE!")
        print(f"  Total rows coded: {rows_completed}")
        print(f"  Total API cost: ${session.state['total_cost']:.2f}")
        print(f"  Output: 31 columns (original + codes + descriptions + enhanced locations)")
        print(f"  Data quality: Expert-validated with historical location preservation")

    log_event(f"Batch {batch_num} complete: {success_count} success, {error_count} errors, "
              f"{quality_issues} quality issues, {checkpoint_count} checkpoints, cost: ${final_cost:.2f}")

if __name__ == "__main__":
    main()